<a href="https://colab.research.google.com/github/Lolit-78/T3-PROYECT/blob/main/Digit_Chatbank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CHATBOT MULTIMODAL

ENTRENAMIENTO DE MODELO DE CLASIFICACIÓN DE INTENSIONES BANCARIAS

In [ ]:
# === 1. Instalar dependencias (solo si no las tienes) ===
# !pip install scikit-learn pandas numpy

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import re
import pickle

# === 2. Cargar dataset y parsear columnas ===
# Leer el archivo como una sola columna inicialmente
try:
    df_raw = pd.read_csv("dataset_bancario_espanol.csv", encoding='utf-8', header=None)
except UnicodeDecodeError:
    try:
        df_raw = pd.read_csv("dataset_bancario_espanol.csv", encoding='latin1', header=None)
    except UnicodeDecodeError:
        df_raw = pd.read_csv("dataset_bancario_espanol.csv", encoding='ISO-8859-1', header=None)

# Expresión regular para extraer el texto y la intención
# Busca cualquier cosa hasta una coma, seguida de comillas, cualquier cosa (la intención) y comillas
pattern = re.compile(r'([^,]*),"(.*)"')

# Aplicar la expresión regular para crear las columnas 'texto' e 'intencion'
def parse_row(row):
    match = pattern.match(row[0])
    if match:
        return pd.Series([match.group(1), match.group(2)])
    return pd.Series([None, None]) # Retorna None si no coincide el patrón

df = df_raw.apply(parse_row, axis=1)
df.columns = ['texto', 'intencion']

# Eliminar filas donde no se pudo extraer texto o intención
df.dropna(subset=['texto', 'intencion'], inplace=True)


print("Shape of dataframe after parsing:", df.shape)
display(df.head()) # Mostrar las primeras filas para verificar

# === 3. Preprocesamiento básico ===
def limpiar_texto(texto):
    # Asegurarse de que 'texto' sea una cadena antes de aplicar lower()
    if isinstance(texto, str):
        texto = texto.lower()
        texto = re.sub(r"[^a-záéíóúñ0-9\s]", "", texto)
        return texto
    return "" # Retorna una cadena vacía si no es una cadena


df["texto"] = df["texto"].apply(limpiar_texto)


# Check if dataframe is empty after dropping rows
if df.empty:
    print("The dataframe is empty after cleaning and dropping rows.")
else:
    # === 4. Vectorización de texto ===
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(df["texto"]).toarray()

    # === 5. Codificación de etiquetas (one-hot) ===
    encoder = OneHotEncoder()
    y = encoder.fit_transform(df[["intencion"]]).toarray()

    # === 6. División en train/test ===
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # === 7. Clase NeuralNetwork (desde cero con NumPy) ===
    def relu(x): return np.maximum(0, x)
    def relu_derivative(x): return (x > 0).astype(float)
    def sigmoid(x): return 1 / (1 + np.exp(-x))
    def tanh(x): return np.tanh(x) # Add tanh activation
    def tanh_derivative(x): return 1.0 - np.tanh(x)**2 # Add tanh derivative

    def xavier_init(size_in, size_out, activation='relu'):
        if activation == 'relu':
            std = np.sqrt(2.0 / size_in)
        elif activation == 'tanh': # Handle tanh initialization
             std = np.sqrt(2.0 / (size_in + size_out)) # Or other tanh-specific init if preferred
        else: # Default for sigmoid or others
            std = np.sqrt(1.0 / size_in)
        return np.random.randn(size_in, size_out) * std

    class NeuralNetwork:
        def __init__(self, layers, activation='relu', seed=42):
            np.random.seed(seed)
            self.layers = layers
            self.activation_name = activation
            self.params = {}
            self.L = len(layers) - 1
            self.loss_history = [] # Add list to store loss history
            for i in range(1, len(layers)):
                act = activation if i < len(layers)-1 else 'sigmoid'
                self.params[f'W{i}'] = xavier_init(layers[i-1], layers[i], activation=act)
                self.params[f'b{i}'] = np.zeros((1, layers[i]))

        def _activation(self, x):
            if self.activation_name == 'relu':
                return relu(x)
            elif self.activation_name == 'sigmoid':
                return sigmoid(x)
            elif self.activation_name == 'tanh':
                 return tanh(x)
            else:
                raise ValueError("Unsupported activation function")

        def _activation_derivative(self, x):
            if self.activation_name == 'relu':
                return relu_derivative(x)
            elif self.activation_name == 'sigmoid':
                return sigmoid(x) * (1 - sigmoid(x))
            elif self.activation_name == 'tanh':
                return tanh_derivative(x)
            else:
                raise ValueError("Unsupported activation function")

        def forward(self, X):
            caches = {'A0': X}
            A = X
            for i in range(1, self.L + 1):
                W, b = self.params[f'W{i}'], self.params[f'b{i}']
                Z = np.dot(A, W) + b
                caches[f'Z{i}'] = Z
                if i == self.L:
                    # Softmax for the output layer
                    expZ = np.exp(Z - np.max(Z, axis=1, keepdims=True))
                    A = expZ / np.sum(expZ, axis=1, keepdims=True)
                else:
                    A = self._activation(Z)
                caches[f'A{i}'] = A
            return A, caches

        def compute_loss(self, Y_hat, Y):
            m = Y.shape[0]
            eps = 1e-9
            # Using Categorical Cross-Entropy loss
            loss = -np.sum(Y * np.log(Y_hat + eps)) / m
            return loss


        def backward(self, caches, X, Y):
            grads = {}
            m = X.shape[0]
            # Gradient for the output layer (Softmax + Cross-Entropy)
            dZ = (caches[f'A{self.L}'] - Y) / m

            for i in range(self.L, 0, -1):
                A_prev = caches[f'A{i-1}']
                grads[f'dW{i}'] = np.dot(A_prev.T, dZ)
                grads[f'db{i}'] = np.sum(dZ, axis=0, keepdims=True)

                if i > 1:
                    # Backpropagate through the activation function of the previous layer
                    W = self.params[f'W{i}']
                    dA_prev = np.dot(dZ, W.T)
                    dZ = dA_prev * self._activation_derivative(caches[f'Z{i-1}'])
            return grads


        def update_params(self, grads, lr):
            for i in range(1, self.L + 1): # Corrected loop range
                self.params[f'W{i}'] -= lr * grads[f'dW{i}']
                self.params[f'b{i}'] -= lr * grads[f'db{i}']


        def fit(self, X, Y, epochs=100, lr=0.01):
            self.loss_history = [] # Reset loss history at the start of fitting
            for epoch in range(epochs):
                Y_hat, caches = self.forward(X)
                loss = self.compute_loss(Y_hat, Y)
                self.loss_history.append(loss) # Store the recorded loss history
                grads = self.backward(caches, X, Y)
                self.update_params(grads, lr)
                if (epoch + 1) % 10 == 0 or epoch == 1:
                    print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss:.4f}")
            return self.loss_history # Return loss history


        def predict(self, X):
            Y_hat, _ = self.forward(X)
            # Return the index of the class with the highest probability
            return np.argmax(Y_hat, axis=1)

    # === 8. Entrenar la red ===
    n_features = X_train.shape[1]
    n_classes = y_train.shape[1]
    nn = NeuralNetwork([n_features, 64, 32, n_classes], activation='relu')
    nn.fit(X_train, y_train, epochs=100, lr=0.01)

    # === 9. Evaluación ===
    y_pred = nn.predict(X_test)
    y_test_labels = np.argmax(y_test, axis=1)

    # Get the unique labels present in the test set
    unique_test_labels = np.unique(y_test_labels)

    print("Accuracy:", accuracy_score(y_test_labels, y_pred))
    # Pass the unique labels from the test set to the labels parameter
    print(classification_report(y_test_labels, y_pred, target_names=encoder.categories_[0], labels=unique_test_labels))
    print("Matriz de confusión:\n", confusion_matrix(y_test_labels, y_pred))

    # === 10. Guardar modelo y preprocesadores ===
    with open("vectorizer.pkl", "wb") as f: pickle.dump(vectorizer, f)
    with open("encoder.pkl", "wb") as f: pickle.dump(encoder, f)
    with open("nn_weights.pkl", "wb") as f: pickle.dump(nn.params, f)

In [ ]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=df)

### Predicciones con el modelo entrenado

In [ ]:
# === Ensure files are saved before loading (for diagnosis) ===
# Check if dataframe is empty before attempting to save
if 'df' in locals() and not df.empty:
    try:
        with open("vectorizer.pkl", "wb") as f: pickle.dump(vectorizer, f)
        with open("encoder.pkl", "wb") as f: pickle.dump(encoder, f)
        with open("nn_weights.pkl", "wb") as f: pickle.dump(nn.params, f)
        print("Model and preprocessor files saved.")
    except Exception as e:
        print(f"Error saving files: {e}")
else:
    print("DataFrame is empty. Skipping file saving.")

# === 1. Cargar modelo y preprocesadores ===
import os
print("Files in current directory:", os.listdir())

try:
    with open("vectorizer.pkl", "rb") as f:
        loaded_vectorizer = pickle.load(f)
    with open("encoder.pkl", "rb") as f:
        loaded_encoder = pickle.load(f)
    with open("nn_weights.pkl", "rb") as f:
        loaded_weights = pickle.load(f)
    print("Model and preprocessor files loaded successfully.")

    # Recreate the neural network structure and load weights
    # Get vocabulary size from the loaded vectorizer safely
    n_features = len(loaded_vectorizer.vocabulary_) if hasattr(loaded_vectorizer, 'vocabulary_') else 0
    # Get number of classes from the loaded encoder safely
    n_classes = loaded_encoder.categories_[0].shape[0] if hasattr(loaded_encoder, 'categories_') and loaded_encoder.categories_ else 0

    if n_features > 0 and n_classes > 0:
        loaded_nn = NeuralNetwork([n_features, 64, 32, n_classes], activation='relu')
        loaded_nn.params = loaded_weights

        # === 2. Preprocess new text ===
        new_text = ["Quiero saber mi saldo actual", "Necesito información sobre un préstamo"]
        cleaned_text = [limpiar_texto(text) for text in new_text]

        # === 3. Vectorize new text ===
        X_new = loaded_vectorizer.transform(cleaned_text).toarray()

        # === 4. Predict intention ===
        predictions = loaded_nn.predict(X_new)

        # === 5. Decode predictions ===
        predicted_intents = loaded_encoder.categories_[0][predictions]

        # === 6. Display results ===
        for text, intent in zip(new_text, predicted_intents):
            print(f"Text: {text} -> Predicted Intent: {intent}")
    else:
        print("Unable to predict: vocabulary or number of classes is zero.")

except FileNotFoundError:
    print("Error: Model or preprocessor files not found after attempting to save in this cell.")
except Exception as e:
    print(f"An unexpected error occurred during loading or prediction: {e}")

ENTRENAMIENTO CON OCR E IMAGENES DE DOCUMENTOS BANCARIOS SIMULADOS

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import os

# Create a directory to save simulated images
if not os.path.exists("simulated_docs"):
    os.makedirs("simulated_docs")

def create_simulated_document(text, filename, img_size=(600, 200), bg_color="white", text_color="black"):
    """Creates a basic simulated document image with text."""
    img = Image.new('RGB', img_size, color = bg_color)
    d = ImageDraw.Draw(img)

    try:
        # Try to use a common font available in Colab, or a default
        font = ImageFont.truetype("LiberationSerif-Regular.ttf", 25)
    except IOError:
        print("Default font not found, using a generic PIL font.")
        font = ImageFont.load_default()

    # Add text wrapping for longer sentences
    words = text.split()
    lines = []
    current_line = ""
    for word in words:
        # Estimate text width (very basic)
        test_line = current_line + word + " "
        if font.getbbox(test_line)[2] < img_size[0] - 40: # Check if line exceeds width (with padding)
            current_line += word + " "
        else:
            lines.append(current_line.strip())
            current_line = word + " "
    lines.append(current_line.strip())

    y_text = 20
    for line in lines:
        d.text((20, y_text), line, fill=text_color, font=font)
        y_text += font.getbbox(line)[3] - font.getbbox(line)[1] + 5 # Move down for next line

    img.save(os.path.join("simulated_docs", filename))
    print(f"Created simulated document: {filename}")

# Define some banking-related texts
banking_texts = [
    "Mi saldo actual es de 1500 dolares.",
    "Quiero transferir 500 a Juan Perez.",
    "Necesito información sobre un préstamo personal.",
    "Pagar mi tarjeta de crédito Visa.",
    "Consultar los movimientos de mi cuenta de ahorros.",
    "Quiero abrir una nueva cuenta corriente."
]

# Create images for each text
for i, text in enumerate(banking_texts):
    create_simulated_document(text, f"document_{i+1}.png")

print("\nSimulated documents created in 'simulated_docs' directory.")

In [ ]:
# === 1. Instalar dependencias para OCR e imágenes ===
!apt-get update
!apt-get install tesseract-ocr tesseract-ocr-spa
!pip install pytesseract Pillow opencv-python

import pytesseract
import cv2
from PIL import Image
import os

In [ ]:
# === 3. Preprocesamiento de imágenes ===
def preprocess_image_for_ocr(image_path):
    """Loads an image and applies basic preprocessing for OCR."""
    # Load the image using OpenCV
    img = cv2.imread(image_path)

    if img is None:
        print(f"Error: Could not load image from {image_path}")
        return None

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply thresholding to get a binary image
    # Adjust the thresholding method and parameters as needed
    _, binary = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    # Optional: Denoising, if needed
    # binary = cv2.fastNlMeansDenoising(binary, None, 30, 7, 21)

    # Optional: Skew correction (more advanced)

    return binary

# Example: Preprocess one of the generated images
# Get the list of simulated document images
image_files = [f for f in os.listdir("simulated_docs") if f.endswith(".png")]

if image_files:
    sample_image_path = os.path.join("simulated_docs", image_files[0])
    preprocessed_img = preprocess_image_for_ocr(sample_image_path)

    if preprocessed_img is not None:
        print(f"Image '{image_files[0]}' preprocessed successfully. Displaying result:")
        # Display the preprocessed image (optional, requires matplotlib or similar)
        # For simple verification, we can just confirm it was processed.
        # To display in Colab, you might need:
        # from google.colab.patches import cv2_imshow
        # cv2_imshow(preprocessed_img)
    else:
        print(f"Failed to preprocess image '{image_files[0]}'.")
else:
    print("No simulated document images found in 'simulated_docs' directory.")

In [ ]:
# === 4. Extracción de texto con OCR ===
def extract_text_from_image(image_path):
    """Extracts text from an image using Tesseract OCR after preprocessing."""
    # Preprocess the image
    preprocessed_img = preprocess_image_for_ocr(image_path)

    if preprocessed_img is None:
        return "Error: Could not preprocess image."

    # Use pytesseract to extract text from the preprocessed image
    # We need to convert the OpenCV image (NumPy array) to a PIL Image for pytesseract
    text = pytesseract.image_to_string(Image.fromarray(preprocessed_img), lang='spa') # Specify Spanish language

    return text

# Example: Extract text from one of the generated images
if image_files:
    sample_image_path = os.path.join("simulated_docs", image_files[0])
    extracted_text = extract_text_from_image(sample_image_path)

    print(f"Extracted text from '{image_files[0]}':")
    print(extracted_text)
else:
    print("No simulated document images found for OCR.")

In [ ]:
# === 5. Limpieza y estructuración del texto extraído ===
def clean_extracted_text(text):
    """Applies basic cleaning to text extracted by OCR."""
    # Ensure text is a string
    if not isinstance(text, str):
        return ""

    # Remove leading/trailing whitespace
    text = text.strip()

    # Replace multiple whitespace characters with a single space
    text = re.sub(r'\s+', ' ', text)

    # Remove characters that might be misread by OCR but are not alphanumeric or common punctuation
    # (Adjust regex based on expected text content and common OCR errors)
    # For now, let's keep it simple and remove non-alphanumeric except spaces and some punctuation
    # text = re.sub(r'[^a-zA-Z0-9\s.,!?;]', '', text) # Example: remove most special chars

    # Convert to lowercase (optional, depending on downstream model)
    # text = text.lower()

    return text

# Example: Clean the text extracted from the sample image
if 'extracted_text' in locals():
    cleaned_extracted_text = clean_extracted_text(extracted_text)
    print(f"Original extracted text:\n{extracted_text}")
    print(f"Cleaned extracted text:\n{cleaned_extracted_text}")
else:
    print("No extracted text found. Please run the OCR extraction step first.")

# Note: For real-world scenarios, structuring the text might involve
# identifying sections (e.g., account number, amount) based on layout analysis or keywords.
# For this basic example, we'll treat the extracted text as a single block.

In [ ]:
# === Step 7: Preparación del conjunto de datos para entrenamiento ===

# Get the list of simulated document images
image_files = sorted([f for f in os.listdir("simulated_docs") if f.endswith(".png")])

# Define the original banking texts/intentions used for generating images
# This list must match the order in which the images were generated (document_1.png -> first text, etc.)
original_banking_texts = [
    "Mi saldo actual es de 1500 pesos.", # Corresponds to document_1.png
    "Quiero transferir 500 a Juan Perez.", # Corresponds to document_2.png
    "Necesito información sobre un préstamo personal.", # Corresponds to document_3.png
    "Pagar mi tarjeta de crédito Visa.", # Corresponds to document_4.png
    "Consultar los movimientos de mi cuenta de ahorros.", # Corresponds to document_5.png
    "Quiero abrir una nueva cuenta corriente." # Corresponds to document_6.png
]

# We need to map these texts to their corresponding intentions as used in the original training data
# Assuming a mapping exists or can be derived from the original df or a separate source
# For this example, let's manually define a simple mapping based on the original training intents
# Correcting keys to match the output of limpiar_texto (removing the final dot)
text_to_intent_mapping = {
    "mi saldo actual es de 1500 pesos": "Consultar saldo",
    "quiero transferir 500 a juan perez": "Transferir dinero",
    "necesito información sobre un préstamo personal": "Solicitar préstamo",
    "pagar mi tarjeta de crédito visa": "Pagar tarjeta de crédito",
    "consultar los movimientos de mi cuenta de ahorros": "Consultar movimientos",
    "quiero abrir una nueva cuenta corriente": "Abrir cuenta"
}
# Note: This mapping assumes the cleaned text will match these keys exactly.
# In a real scenario, you might need more flexible matching or store intentions directly with images.

extracted_data = []

# Process each simulated image
for i, image_file in enumerate(image_files):
    image_path = os.path.join("simulated_docs", image_file)

    # Preprocess, extract text, and clean
    extracted_text = extract_text_from_image(image_path)
    cleaned_text = clean_extracted_text(extracted_text)

    # Get the original text used to generate this image (for mapping to intent)
    # Need to clean the original text too to match the cleaned extracted text for mapping
    original_cleaned_text = limpiar_texto(original_banking_texts[i])


    # Find the corresponding intent using the mapping
    # Use the cleaned original text as the key for robustness
    corresponding_intent = text_to_intent_mapping.get(original_cleaned_text, "Unknown Intent")

    extracted_data.append({"image_file": image_file, "extracted_text": cleaned_text, "original_text": original_banking_texts[i], "intent": corresponding_intent})

# Create a DataFrame
df_simulated_ocr = pd.DataFrame(extracted_data)

print("DataFrame created from simulated images:")
display(df_simulated_ocr.head())

### Vectorize Extracted Text

In [ ]:
# === Step 8: Vectorización del texto extraído ===

# Use the existing TfidfVectorizer or create a new one if needed
# For this pipeline, we'll fit a new vectorizer on the OCR-extracted text
# to create features specific to the text extracted from images.
ocr_vectorizer = TfidfVectorizer()
X_ocr = ocr_vectorizer.fit_transform(df_simulated_ocr["extracted_text"]).toarray()

# We also need to encode the intentions from the simulated data
ocr_encoder = OneHotEncoder()
y_ocr = ocr_encoder.fit_transform(df_simulated_ocr[["intent"]]).toarray()

print("Shape of vectorized OCR text features (X_ocr):", X_ocr.shape)
print("Shape of one-hot encoded OCR intentions (y_ocr):", y_ocr.shape)

# For training a model, you would typically split this OCR-based dataset
# into training and testing sets, similar to the original text data pipeline.
# X_ocr_train, X_ocr_test, y_ocr_train, y_ocr_test = train_test_split(X_ocr, y_ocr, test_size=0.3, random_state=42)

### Train Classification Model on OCR Data

In [ ]:
# === Step 9: Diseño y entrenamiento del modelo de clasificación ===

# Split the OCR-based dataset into training and testing sets
X_ocr_train, X_ocr_test, y_ocr_train, y_ocr_test = train_test_split(X_ocr, y_ocr, test_size=0.3, random_state=42)

# Initialize and train the neural network model on the OCR data
# Use the same NeuralNetwork class defined previously
n_features_ocr = X_ocr_train.shape[1]
n_classes_ocr = y_ocr_train.shape[1]
nn_ocr = NeuralNetwork([n_features_ocr, 64, 32, n_classes_ocr], activation='relu')

print("Starting training on OCR data...")
nn_ocr.fit(X_ocr_train, y_ocr_train, epochs=100, lr=0.01)
print("Training on OCR data finished.")

# Note: For a real-world scenario with limited data like this,
# you might consider data augmentation on images or using transfer learning
# with a pre-trained model, but we'll stick to the simple NN for this example.

### Evaluate Classification Model on OCR Data

In [ ]:
# === Step 10: Evaluación del modelo entrenado con datos de OCR ===

# Make predictions on the test set
y_ocr_pred = nn_ocr.predict(X_ocr_test)

# Get the true labels for the test set
y_ocr_test_labels = np.argmax(y_ocr_test, axis=1)

# Get the unique labels present in the OCR test set for the classification report
unique_ocr_test_labels = np.unique(y_ocr_test_labels)

print("Evaluation on OCR Data:")
print("Accuracy:", accuracy_score(y_ocr_test_labels, y_ocr_pred))

# Get the target names from the OCR encoder
ocr_target_names = ocr_encoder.categories_[0]

# Print the classification report, specifying labels present in the test set
print(classification_report(y_ocr_test_labels, y_ocr_pred, target_names=ocr_target_names, labels=unique_ocr_test_labels, zero_division=0))

print("Confusion Matrix:\n", confusion_matrix(y_ocr_test_labels, y_ocr_pred, labels=unique_ocr_test_labels))

### Implement End-to-End Prediction Pipeline for Images

In [ ]:
# === Step 11: Implementación de la predicción con imágenes ===

def predict_intention_from_image(image_path, vectorizer, encoder, neural_network):
    """
    Predicts the banking intention from a document image.

    Args:
        image_path (str): Path to the document image file.
        vectorizer (TfidfVectorizer): Trained TF-IDF vectorizer.
        encoder (OneHotEncoder): Trained One-Hot Encoder for intentions.
        neural_network (NeuralNetwork): Trained neural network model.

    Returns:
        str: Predicted banking intention.
        str: Extracted and cleaned text from the image.
    """
    # 1. Preprocess the image
    preprocessed_img = preprocess_image_for_ocr(image_path)

    if preprocessed_img is None:
        return "Prediction Error: Could not preprocess image.", ""

    # 2. Extract text with OCR
    extracted_text = pytesseract.image_to_string(Image.fromarray(preprocessed_img), lang='spa')

    # 3. Clean the extracted text
    cleaned_text = clean_extracted_text(extracted_text)

    if not cleaned_text:
        return "Prediction Error: No meaningful text extracted from image.", cleaned_text

    # 4. Vectorize the cleaned text
    # Use the *fitted* vectorizer to transform the new text
    X_new = vectorizer.transform([cleaned_text]).toarray() # transform expects an iterable

    # 5. Predict intention
    predictions = neural_network.predict(X_new)

    # 6. Decode prediction
    predicted_intent = encoder.categories_[0][predictions][0] # Get the single predicted label

    return predicted_intent, cleaned_text

# === Step 12: Pruebas con imágenes simuladas ===
print("Testing the end-to-end pipeline on simulated images:")

# Get the list of simulated document images
image_files_for_test = sorted([f for f in os.listdir("simulated_docs") if f.endswith(".png")])

if image_files_for_test:
    # Use the trained OCR-based model and preprocessors
    # Assuming nn_ocr, ocr_vectorizer, and ocr_encoder are available from previous steps
    if 'nn_ocr' in locals() and 'ocr_vectorizer' in locals() and 'ocr_encoder' in locals():
        for image_file in image_files_for_test:
            image_path = os.path.join("simulated_docs", image_file)
            predicted_intent, extracted_cleaned_text = predict_intention_from_image(image_path, ocr_vectorizer, ocr_encoder, nn_ocr)
            print(f"Image: {image_file} -> Extracted Text: '{extracted_cleaned_text}' -> Predicted Intent: {predicted_intent}")
    else:
        print("Error: OCR model (nn_ocr), vectorizer (ocr_vectorizer), or encoder (ocr_encoder) not found. Please run training steps first.")
else:
    print("No simulated document images found in 'simulated_docs' directory for testing.")

In [ ]:
# =========================
# 1. INSTALACIÓN DE LIBRERÍAS
# =========================
!pip install pillow faker pytesseract opencv-python pandas

import os
import cv2
import pytesseract
from PIL import Image, ImageDraw, ImageFont
from faker import Faker
import random
import pandas as pd

# Configuración de Faker para español
fake = Faker('es_ES')

# Directorios
os.makedirs("documentos_img", exist_ok=True)
os.makedirs("procesados", exist_ok=True)

# =========================
# 2. GENERAR DOCUMENTOS SIMULADOS
# =========================
def generar_documento(tipo, idx):
    ancho, alto = 800, 400
    img = Image.new('RGB', (ancho, alto), color='white')
    draw = ImageDraw.Draw(img)

    # Fuente
    try:
        font_title = ImageFont.truetype("arial.ttf", 30)
        font_text = ImageFont.truetype("arial.ttf", 20)
    except:
        font_title = ImageFont.load_default()
        font_text = ImageFont.load_default()

    # Datos simulados
    nombre = fake.name()
    fecha = fake.date_between(start_date='-1y', end_date='today').strftime("%d/%m/%Y")
    valor = f"${random.randint(100, 5000):,}.{random.randint(0,99):02}".replace(",", ".")
    numero_doc = f"{tipo[:2].upper()}-{random.randint(100000,999999)}"
    banco = "Banco Digital"

    # Encabezado
    draw.text((20, 20), banco, fill="black", font=font_title)
    draw.text((20, 60), f"Tipo de documento: {tipo}", fill="black", font=font_text)
    draw.text((20, 90), f"Número: {numero_doc}", fill="black", font=font_text)

    # Datos principales
    draw.text((20, 140), f"Nombre del cliente: {nombre}", fill="black", font=font_text)
    draw.text((20, 180), f"Fecha: {fecha}", fill="black", font=font_text)
    draw.text((20, 220), f"Valor: {valor} USD", fill="black", font=font_text)

    # Firma simulada
    draw.text((20, 300), "Firma:", fill="black", font=font_text)
    draw.line((100, 320, 300, 280), fill="black", width=2)

    # Guardar imagen
    filename = f"documentos_img/{tipo}_{idx}.png"
    img.save(filename)

    return {
        "tipo_documento": tipo,
        "nombre_cliente": nombre,
        "fecha": fecha,
        "valor": valor,
        "numero_documento": numero_doc,
        "banco": banco,
        "ruta_imagen": filename
    }

# Tipos de documentos
tipos_documentos = ["Cheque", "Estado de cuenta", "Recibo de depósito", "Comprobante de transferencia"]

# Generar dataset de imágenes
datos_generados = []
for tipo in tipos_documentos:
    for i in range(20):  # 20 por tipo
        datos_generados.append(generar_documento(tipo, i+1))

df = pd.DataFrame(datos_generados)
df.to_csv("procesados/dataset_original.csv", index=False)

print("✅ Imágenes generadas y datos guardados en 'procesados/dataset_original.csv'")

# =========================
# 3. OCR PARA EXTRAER DATOS
# =========================
pytesseract.pytesseract.tesseract_cmd = r"/usr/bin/tesseract"  # Ruta en Google Colab

def extraer_datos_ocr(ruta_img):
    img = cv2.imread(ruta_img)

    # Preprocesamiento
    gris = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, umbral = cv2.threshold(gris, 150, 255, cv2.THRESH_BINARY)

    # OCR
    texto = pytesseract.image_to_string(umbral, lang='spa')

    return texto

# Procesar imágenes y guardar texto
resultados_ocr = []
for doc in datos_generados:
    texto_extraido = extraer_datos_ocr(doc["ruta_imagen"])
    resultados_ocr.append({
        "ruta_imagen": doc["ruta_imagen"],
        "texto_ocr": texto_extraido
    })

df_ocr = pd.DataFrame(resultados_ocr)
df_ocr.to_csv("procesados/resultados_ocr.csv", index=False)

print("✅ OCR completado y guardado en 'procesados/resultados_ocr.csv'")

### Vectorize Extracted Text from 80 Simulated Images

In [ ]:
# === Step 8: Vectorización del texto extraído (de 80 imágenes) ===

# Load the OCR results DataFrame
import pandas as pd
df_ocr = pd.read_csv("procesados/resultados_ocr.csv")

# Handle potential missing values in text column before vectorization
df_ocr.dropna(subset=['texto_ocr'], inplace=True)
df_ocr['texto_ocr'] = df_ocr['texto_ocr'].fillna('') # Replace NaN with empty string


# Apply TF-IDF vectorization
# For this larger dataset, we'll fit a new vectorizer
large_ocr_vectorizer = TfidfVectorizer()
X_large_ocr = large_ocr_vectorizer.fit_transform(df_ocr["texto_ocr"]).toarray()

print("Shape of vectorized OCR text features (X_large_ocr):", X_large_ocr.shape)

# We will need the corresponding labels from the original generated data for training
# This will be addressed in the next step (preparing training data)

### Prepare Combined Dataset for Training

In [ ]:
# === Step 7 (Revisited): Preparar conjunto de datos combinado ===

# Load the OCR results DataFrame
df_ocr = pd.read_csv("procesados/resultados_ocr.csv")

# Load the original dataset with labels
df_original = pd.read_csv("procesados/dataset_original.csv")

# Merge the two dataframes based on the image path
# Ensure the column names used for merging match in both dataframes
df_combined = pd.merge(df_ocr, df_original[['ruta_imagen', 'tipo_documento']], on='ruta_imagen')

# Rename 'tipo_documento' to 'intencion' for consistency with the original text data
df_combined = df_combined.rename(columns={'tipo_documento': 'intencion'})

# Clean the extracted text again, just in case (optional, but good practice)
# Use the existing limpiar_texto function or create a new one if needed
# For consistency, let's use the existing one, assuming it's suitable for OCR output
def limpiar_texto_ocr(texto):
    # Ensure text is a string
    if not isinstance(texto, str):
        return ""
    texto = texto.lower()
    # Keep a broader range of characters for OCR output initially
    texto = re.sub(r"[^a-záéíóúñ0-9\s.,!?;:]", "", texto) # Example: keep some punctuation
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto


df_combined["texto_ocr_cleaned"] = df_combined["texto_ocr"].apply(limpiar_texto_ocr)

# Verify the combined dataframe
print("Combined DataFrame shape:", df_combined.shape)
display(df_combined.head())

# Now, vectorize the cleaned OCR text from the combined dataframe
# Use a new vectorizer fitted on this combined data, or the one from Step 8 if preferred
# Using the vectorizer from Step 8 is more consistent if we want to use its vocabulary
if 'large_ocr_vectorizer' in locals():
    X_combined_ocr = large_ocr_vectorizer.transform(df_combined["texto_ocr_cleaned"]).toarray()
    print("Shape of vectorized combined OCR text features (X_combined_ocr) using existing vectorizer:", X_combined_ocr.shape)
else:
    # If the vectorizer from Step 8 is not available, fit a new one
    combined_ocr_vectorizer = TfidfVectorizer()
    X_combined_ocr = combined_ocr_vectorizer.fit_transform(df_combined["texto_ocr_cleaned"]).toarray()
    large_ocr_vectorizer = combined_ocr_vectorizer # Update the variable for later use
    print("Shape of vectorized combined OCR text features (X_combined_ocr) using a new vectorizer:", X_combined_ocr.shape)


# Encode the intention labels from the combined dataframe
combined_ocr_encoder = OneHotEncoder()
y_combined_ocr = combined_ocr_encoder.fit_transform(df_combined[["intencion"]]).toarray()

print("Shape of one-hot encoded combined intentions (y_combined_ocr):", y_combined_ocr.shape)

### Train Classification Model on Combined OCR Data

In [ ]:
# === Step 9: Diseño y entrenamiento del modelo de clasificación (con datos combinados) ===

# Split the combined OCR-based dataset into training and testing sets
X_combined_ocr_train, X_combined_ocr_test, y_combined_ocr_train, y_combined_ocr_test = train_test_split(X_combined_ocr, y_combined_ocr, test_size=0.3, random_state=42)

# Initialize and train the neural network model on the combined OCR data
n_features_combined_ocr = X_combined_ocr_train.shape[1]
n_classes_combined_ocr = y_combined_ocr_train.shape[1]
nn_combined_ocr = NeuralNetwork([n_features_combined_ocr, 64, 32, n_classes_combined_ocr], activation='relu')

print("Starting training on combined OCR data...")
nn_combined_ocr.fit(X_combined_ocr_train, y_combined_ocr_train, epochs=100, lr=0.01)
print("Training on combined OCR data finished.")

### Evaluate Classification Model on Combined OCR Data

In [ ]:
# === Step 10: Evaluación del modelo entrenado con datos combinados de OCR ===

# Make predictions on the test set
y_combined_ocr_pred = nn_combined_ocr.predict(X_combined_ocr_test)

# Get the true labels for the test set
y_combined_ocr_test_labels = np.argmax(y_combined_ocr_test, axis=1)

# Get the unique labels present in the combined OCR test set for the classification report
unique_combined_ocr_test_labels = np.unique(y_combined_ocr_test_labels)

print("Evaluation on Combined OCR Data:")
print("Accuracy:", accuracy_score(y_combined_ocr_test_labels, y_combined_ocr_pred))

# Get the target names from the combined OCR encoder
combined_ocr_target_names = combined_ocr_encoder.categories_[0]

# Print the classification report, specifying labels present in the test set
print(classification_report(y_combined_ocr_test_labels, y_combined_ocr_pred, target_names=combined_ocr_target_names, labels=unique_combined_ocr_test_labels, zero_division=0))

print("Confusion Matrix:\n", confusion_matrix(y_combined_ocr_test_labels, y_combined_ocr_pred, labels=unique_combined_ocr_test_labels))

### Implement End-to-End Prediction Pipeline for Images (using combined data model)

In [ ]:
# === Step 11: Implementación de la predicción con imágenes (usando el modelo entrenado con datos combinados) ===

def predict_intention_from_image_combined(image_path, vectorizer, encoder, neural_network):
    """
    Predicts the banking intention from a document image using the model trained on combined OCR data.

    Args:
        image_path (str): Path to the document image file.
        vectorizer (TfidfVectorizer): Trained TF-IDF vectorizer (fitted on combined OCR data).
        encoder (OneHotEncoder): Trained One-Hot Encoder for intentions (fitted on combined OCR intentions).
        neural_network (NeuralNetwork): Trained neural network model (trained on combined OCR data).

    Returns:
        str: Predicted banking intention.
        str: Extracted and cleaned text from the image.
    """
    # 1. Preprocess the image
    preprocessed_img = preprocess_image_for_ocr(image_path)

    if preprocessed_img is None:
        return "Prediction Error: Could not preprocess image.", ""

    # 2. Extract text with OCR
    extracted_text = pytesseract.image_to_string(Image.fromarray(preprocessed_img), lang='spa')

    # 3. Clean the extracted text
    # Use the specific cleaning function for OCR text if different, otherwise use the generic one
    cleaned_text = limpiar_texto_ocr(extracted_text) # Using the cleaning function defined for OCR text

    if not cleaned_text:
        return "Prediction Error: No meaningful text extracted from image.", cleaned_text

    # 4. Vectorize the cleaned text
    # Use the *fitted* vectorizer to transform the new text
    X_new = vectorizer.transform([cleaned_text]).toarray() # transform expects an iterable

    # 5. Predict intention
    predictions = neural_network.predict(X_new)

    # 6. Decode prediction
    predicted_intent = encoder.categories_[0][predictions][0] # Get the single predicted label

    return predicted_intent, cleaned_text

# === Step 12: Pruebas con imágenes simuladas (usando el modelo entrenado con datos combinados) ===
print("Testing the end-to-end pipeline on simulated images (using combined data model):")

# Get the list of simulated document images
image_files_for_test = sorted([f for f in os.listdir("documentos_img") if f.endswith(".png")]) # Use the directory with 80 images

if image_files_for_test:
    # Use the trained combined OCR model and preprocessors
    # Assuming nn_combined_ocr, large_ocr_vectorizer (or combined_ocr_vectorizer), and combined_ocr_encoder are available
    if ('nn_combined_ocr' in locals() or 'nn_combined_ocr' in globals()) and \
       (('large_ocr_vectorizer' in locals() or 'large_ocr_vectorizer' in globals()) or \
        ('combined_ocr_vectorizer' in locals() or 'combined_ocr_vectorizer' in globals())) and \
       ('combined_ocr_encoder' in locals() or 'combined_ocr_encoder' in globals()):

        # Determine which vectorizer variable is available
        current_vectorizer = large_ocr_vectorizer if 'large_ocr_vectorizer' in locals() or 'large_ocr_vectorizer' in globals() else combined_ocr_vectorizer

        for image_file in image_files_for_test:
            image_path = os.path.join("documentos_img", image_file)
            predicted_intent, extracted_cleaned_text = predict_intention_from_image_combined(image_path, current_vectorizer, combined_ocr_encoder, nn_combined_ocr)
            print(f"Image: {image_file} -> Extracted Text: '{extracted_cleaned_text}' -> Predicted Intent: {predicted_intent}")
    else:
        print("Error: Combined OCR model (nn_combined_ocr), vectorizer, or encoder not found. Please run training steps first.")
else:
    print("No simulated document images found in 'documentos_img' directory for testing.")

# EXPERIMENTACION
Realiza un proceso de experimentación simple y fácil de comprender para comparar diferentes arquitecturas de redes neuronales, funciones de activación e hiperparámetros para la clasificación de intenciones bancarias basadas en texto y OCR, utilizando como baseline una regresión logística. Guarda los resultados de rendimiento y las curvas de entrenamiento.
Asegurarse de tener listos los datos vectorizados y codificados del OCR (X_combined_ocr, y_combined_ocr) y dividirlos en conjuntos de entrenamiento y prueba.

In [ ]:
# 1. Verify variables and their shapes
print("Shape of X_combined_ocr:", X_combined_ocr.shape if 'X_combined_ocr' in locals() or 'X_combined_ocr' in globals() else "X_combined_ocr not found")
print("Shape of y_combined_ocr:", y_combined_ocr.shape if 'y_combined_ocr' in locals() or 'y_combined_ocr' in globals() else "y_combined_ocr not found")

# Check if variables exist and are not empty before splitting
if ('X_combined_ocr' in locals() or 'X_combined_ocr' in globals()) and \
   ('y_combined_ocr' in locals() or 'y_combined_ocr' in globals()) and \
   X_combined_ocr.shape[0] > 0 and y_combined_ocr.shape[0] > 0:

    # 3. Split the data into training and testing sets
    X_combined_ocr_train, X_combined_ocr_test, y_combined_ocr_train, y_combined_ocr_test = train_test_split(
        X_combined_ocr, y_combined_ocr, test_size=0.3, random_state=42
    )

    # 4. Print the shapes of the resulting sets
    print("\nShapes after splitting:")
    print("X_combined_ocr_train shape:", X_combined_ocr_train.shape)
    print("X_combined_ocr_test shape:", X_combined_ocr_test.shape)
    print("y_combined_ocr_train shape:", y_combined_ocr_train.shape)
    print("y_combined_ocr_test shape:", y_combined_ocr_test.shape)

else:
    print("\nX_combined_ocr or y_combined_ocr not found or are empty. Cannot split data.")

## Definir configuraciones de red neuronal a probar

### Subtask:
Especificar claramente las 3 arquitecturas (número de capas y neuronas) y las funciones de activación (ReLU, Sigmoid, Tanh) que se experimentarán. También definir los hiperparámetros (learning rate, epochs) para cada prueba.

In [ ]:
# Get input and output dimensions from the prepared data
input_dim = X_combined_ocr_train.shape[1]
output_dim = y_combined_ocr_train.shape[1]

# Define the neural network configurations for experimentation
nn_experiments = [
    {
        'name': 'Simple_NN_ReLU',
        'architecture': [input_dim, 32, output_dim], # Simple architecture: input -> 32 neurons -> output
        'activation': 'relu',
        'lr': 0.01,
        'epochs': 100
    },
    {
        'name': 'Medium_NN_ReLU',
        'architecture': [input_dim, 64, 32, output_dim], # Medium architecture: input -> 64 -> 32 -> output
        'activation': 'relu',
        'lr': 0.01,
        'epochs': 100
    },
    {
        'name': 'Medium_NN_Sigmoid',
        'architecture': [input_dim, 64, 32, output_dim], # Same medium architecture, but with Sigmoid activation
        'activation': 'sigmoid', # Note: Sigmoid should be used for output, but this config tests it for hidden layers
        'lr': 0.01,
        'epochs': 100
    },
     {
        'name': 'Medium_NN_Tanh',
        'architecture': [input_dim, 64, 32, output_dim], # Same medium architecture, but with Tanh activation
        'activation': 'tanh',
        'lr': 0.01,
        'epochs': 100
    },
    {
        'name': 'Medium_NN_ReLU_HigherLR',
        'architecture': [input_dim, 64, 32, output_dim], # Medium architecture with a higher learning rate
        'activation': 'relu',
        'lr': 0.05, # Higher learning rate
        'epochs': 100
    }
]

print("Defined Neural Network Experiment Configurations:")
for exp in nn_experiments:
    print(f"- Name: {exp['name']}, Architecture: {exp['architecture']}, Activation: {exp['activation']}, LR: {exp['lr']}, Epochs: {exp['epochs']}")

## Implementar y entrenar el modelo baseline (regresión logística)

### Subtask:
Configurar y entrenar un modelo de Regresión Logística de scikit-learn con los mismos datos de entrenamiento (`X_combined_ocr_train`, `y_combined_ocr_train`).

In [ ]:
from sklearn.linear_model import LogisticRegression

# Convert one-hot encoded training labels back to single-column labels for scikit-learn
y_combined_ocr_train_labels = np.argmax(y_combined_ocr_train, axis=1)

# 2. Instantiate a LogisticRegression model
# Use 'auto' for multi_class and 'liblinear' as a suitable solver for small datasets
# Increase max_iter if convergence issues arise
log_reg_model = LogisticRegression(multi_class='auto', solver='liblinear', random_state=42, max_iter=1000)

# 3. Train the LogisticRegression model
print("Starting training of Logistic Regression model...")
log_reg_model.fit(X_combined_ocr_train, y_combined_ocr_train_labels)
print("Training of Logistic Regression model finished.")

# 4. The trained model is stored in the 'log_reg_model' variable.

## Evaluar el baseline

### Subtask:
Evaluate the trained Logistic Regression model on the test set (`X_combined_ocr_test`, `y_combined_ocr_test`) and record its performance metrics.

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Convert one-hot encoded test labels back to single-column labels
y_combined_ocr_test_labels = np.argmax(y_combined_ocr_test, axis=1)

# 5. Make predictions on the test set
y_combined_ocr_log_reg_pred = log_reg_model.predict(X_combined_ocr_test)

# 6. Evaluate the model
accuracy = accuracy_score(y_combined_ocr_test_labels, y_combined_ocr_log_reg_pred)
report = classification_report(y_combined_ocr_test_labels, y_combined_ocr_log_reg_pred, target_names=combined_ocr_encoder.categories_[0], zero_division=0)
conf_matrix = confusion_matrix(y_combined_ocr_test_labels, y_combined_ocr_log_reg_pred, labels=np.unique(y_combined_ocr_test_labels))

print("Evaluation of Logistic Regression Model on Combined OCR Data:")
print("Accuracy:", accuracy)
print("Classification Report:\n", report)
print("Confusion Matrix:\n", conf_matrix)

# Store the results
log_reg_results = {
    'model_name': 'Logistic Regression',
    'accuracy': accuracy,
    'classification_report': report,
    'confusion_matrix': conf_matrix.tolist() # Convert numpy array to list for easier storage/viewing
}

print("\nLogistic Regression results stored.")

## Entrenar y evaluar cada configuración de red neuronal

### Subtask:
Iterar sobre las configuraciones de red neuronal definidas en `nn_experiments`. Para cada configuración, inicializar la red, entrenarla y evaluarla, registrando la pérdida por epoch y las métricas de rendimiento.

In [ ]:
# Initialize list to store experiment results
nn_experiment_results = []

# Loop through each configuration
for exp in nn_experiments:
    print(f"\n--- Training Experiment: {exp['name']} ---")
    print(f"  Architecture: {exp['architecture']}, Activation: {exp['activation']}, LR: {exp['lr']}, Epochs: {exp['epochs']}")

    # Instantiate the neural network
    nn = NeuralNetwork(layers=exp['architecture'], activation=exp['activation'], seed=42)

    # Train the neural network and get loss history
    # Assuming X_combined_ocr_train and y_combined_ocr_train are available
    loss_history = nn.fit(X_combined_ocr_train, y_combined_ocr_train, epochs=exp['epochs'], lr=exp['lr'])

    print(f"  Training complete for {exp['name']}.")

    # Evaluate the model
    # Assuming X_combined_ocr_test and y_combined_ocr_test are available
    y_combined_ocr_test_labels = np.argmax(y_combined_ocr_test, axis=1)

    # Make sure y_combined_ocr_pred is computed correctly before passing to metrics
    y_combined_ocr_pred = nn.predict(X_combined_ocr_test)

    # Get the unique labels present in the test set for the classification report
    unique_combined_ocr_test_labels = np.unique(y_combined_ocr_test_labels)
    # Get the target names from the encoder (assuming combined_ocr_encoder is available)
    combined_ocr_target_names = combined_ocr_encoder.categories_[0]


    accuracy = accuracy_score(y_combined_ocr_test_labels, y_combined_ocr_pred)
    report = classification_report(y_combined_ocr_test_labels, y_combined_ocr_pred, target_names=combined_ocr_target_names, labels=unique_combined_ocr_test_labels, zero_division=0)
    conf_matrix = confusion_matrix(y_combined_ocr_test_labels, y_combined_ocr_pred, labels=unique_combined_ocr_test_labels)

    print(f"\n  Evaluation for {exp['name']}:")
    print(f"  Accuracy: {accuracy:.4f}")
    print("  Classification Report:\n", report)
    print("  Confusion Matrix:\n", conf_matrix)

    # Store the results
    nn_experiment_results.append({
        'name': exp['name'],
        'architecture': exp['architecture'],
        'activation': exp['activation'],
        'lr': exp['lr'],
        'epochs': exp['epochs'],
        'accuracy': accuracy,
        'classification_report': report,
        'confusion_matrix': conf_matrix.tolist(), # Convert numpy array to list
        'loss_history': loss_history # Store the recorded loss history
    })

print("\n--- All Neural Network Experiments Complete ---")
print(f"Results stored for {len(nn_experiment_results)} experiments.")

## Recopilar y guardar todos los resultados

### Subtask:
Consolidar las métricas de rendimiento de la Regresión Logística y de todas las configuraciones de Red Neuronal probadas. Guardar estos resultados y las historias de pérdida (por ejemplo, en DataFrames o archivos pickle).

In [ ]:
import pickle
import pandas as pd

# 1. Create a dictionary to hold all experiment results
all_experiment_results = {}

# 2. Add the log_reg_results dictionary
# Assuming 'log_reg_results' is available from a previous step
if 'log_reg_results' in locals() or 'log_reg_results' in globals():
    all_experiment_results['Logistic Regression'] = log_reg_results
    print("Added Logistic Regression results to consolidated results.")
else:
    print("Warning: 'log_reg_results' not found. Skipping adding Logistic Regression results.")


# 3. Add the nn_experiment_results list
# Assuming 'nn_experiment_results' is available from a previous step
if 'nn_experiment_results' in locals() or 'nn_experiment_results' in globals():
    all_experiment_results['Neural Networks'] = nn_experiment_results
    print(f"Added {len(nn_experiment_results)} Neural Network experiment results to consolidated results.")
else:
    print("Warning: 'nn_experiment_results' not found. Skipping adding Neural Network results.")


# 4. Optionally, convert the nn_experiment_results list into a pandas DataFrame
if 'nn_experiment_results' in locals() or 'nn_experiment_results' in globals():
    # Create a list of dictionaries without the loss history for the DataFrame
    nn_metrics_list = []
    for result in nn_experiment_results:
        metrics_dict = result.copy()
        # Remove loss_history as it's not suitable for a simple metrics DataFrame
        if 'loss_history' in metrics_dict:
            del metrics_dict['loss_history']
        nn_metrics_list.append(metrics_dict)

    if nn_metrics_list:
        df_nn_metrics = pd.DataFrame(nn_metrics_list)
        print("\nCreated DataFrame for Neural Network experiment metrics.")
        display(df_nn_metrics)
    else:
        df_nn_metrics = pd.DataFrame() # Create empty DataFrame if list is empty
        print("\nNeural Network metrics list is empty. No DataFrame created.")
else:
    df_nn_metrics = pd.DataFrame() # Ensure df_nn_metrics is defined even if no NN results


# 5. Save the complete results dictionary to a file using pickle
try:
    with open("experiment_results.pkl", "wb") as f:
        pickle.dump(all_experiment_results, f)
    print("\nComplete experiment results saved to 'experiment_results.pkl'.")
except Exception as e:
    print(f"\nError saving 'experiment_results.pkl': {e}")


# 6. Save the DataFrame of NN results (if created and not empty) to a CSV file
if not df_nn_metrics.empty:
    try:
        df_nn_metrics.to_csv("nn_experiment_metrics.csv", index=False)
        print("Neural Network experiment metrics saved to 'nn_experiment_metrics.csv'.")
    except Exception as e:
        print(f"\nError saving 'nn_experiment_metrics.csv': {e}")
else:
    print("\nNo Neural Network metrics DataFrame to save to CSV.")


# 7. Print a confirmation message (already included in steps 5 and 6)

## Comparar y visualizar resultados

### Subtask:
Presentar una comparación clara del rendimiento de todos los modelos (Regresión Logística y las diferentes RN) utilizando tablas o gráficos sencillos (ej. gráfico de barras de precisión o F1-score). Visualizar las curvas de pérdida de las redes neuronales.

In [ ]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# 1. Load the experiment results
try:
    with open("experiment_results.pkl", "rb") as f:
        all_experiment_results = pickle.load(f)
    print("Experiment results loaded successfully.")
except FileNotFoundError:
    print("Error: experiment_results.pkl not found. Please ensure the previous step to save results was completed.")
    all_experiment_results = None # Set to None to avoid errors in subsequent steps

# Check if results were loaded
if all_experiment_results:
    # 2. Extract Logistic Regression and Neural Network results
    log_reg_result = all_experiment_results.get('Logistic Regression')
    nn_results_list = all_experiment_results.get('Neural Networks', [])

    # 3. Create a pandas DataFrame from the Neural Network results
    # Exclude loss_history for the comparison table
    nn_comparison_list = []
    for result in nn_results_list:
        nn_comparison_list.append({
            'name': result['name'],
            'accuracy': result['accuracy']
        })

    df_comparison = pd.DataFrame(nn_comparison_list)

    # 4. Add Logistic Regression results to the DataFrame
    if log_reg_result:
        log_reg_comparison = {'name': log_reg_result['model_name'], 'accuracy': log_reg_result['accuracy']}
        df_comparison = pd.concat([df_comparison, pd.DataFrame([log_reg_comparison])], ignore_index=True)
        print("\nComparison data prepared, including Logistic Regression.")
    else:
        print("\nWarning: Logistic Regression results not found. Comparison table will only contain NN results.")

    # 5. Sort the DataFrame by accuracy
    df_comparison_sorted = df_comparison.sort_values(by='accuracy', ascending=False)

    print("\nPerformance Comparison Table:")
    display(df_comparison_sorted)

    # 6. Create a bar plot comparing accuracy
    plt.figure(figsize=(12, 6))
    sns.barplot(x='name', y='accuracy', data=df_comparison_sorted, palette='viridis')
    plt.ylabel('Accuracy')
    plt.xlabel('Model')
    plt.title('Model Accuracy Comparison')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

    # 7. and 8. Plot loss history for Neural Networks
    if nn_results_list:
        plt.figure(figsize=(12, 6))
        for result in nn_results_list:
            if 'loss_history' in result and result['loss_history']:
                plt.plot(result['loss_history'], label=result['name'])
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title('Neural Network Training Loss History')
        plt.legend()
        plt.grid(True)
        plt.show()
    else:
        print("\nNo Neural Network loss history available to plot.")

else:
    print("Cannot proceed with comparison and plotting as experiment results were not loaded.")

# Task
Implementar una pipeline multimodal simple en Python que pueda procesar tanto texto directo como imágenes de documentos bancarios simulados ("dataset_bancario_espanol.csv"), utilizando los modelos de clasificación de intenciones y OCR previamente desarrollados para predecir la intención del usuario a partir de la entrada, y proporcionar una respuesta básica basada en la intención predicha.

## Definir los tipos de entrada

### Subtask:
Establecer claramente que el sistema puede recibir dos tipos principales de entrada: texto directo o una imagen de documento.


In [ ]:
# This is a placeholder cell to ensure the next command is a Markdown cell.
# It will be deleted.
pass

**Reasoning**:
Create a new Markdown cell and explain the two input types for the multimodal pipeline as requested in the subtask.



In [ ]:
# %% [markdown]
# ## Pipeline Multimodal: Tipos de Entrada
#
# El sistema de pipeline multimodal está diseñado para procesar la entrada del usuario de dos maneras principales:
#
# 1.  **Entrada de Texto Directo:** El usuario puede escribir directamente su consulta o solicitud como texto.
#
# 2.  **Entrada de Imagen de Documento:** El usuario puede proporcionar una imagen de un documento bancario simulado.
#
# En la implementación, la pipeline diferenciará entre estos dos tipos de entrada típicamente revisando el formato o tipo de la entrada proporcionada. Por ejemplo, una función de procesamiento podría aceptar una cadena (para texto directo) o una ruta de archivo/objeto de imagen (para una imagen), utilizando lógica condicional para aplicar el flujo de procesamiento adecuado (procesamiento de texto o procesamiento con OCR).

## Manejar la entrada de texto

### Subtask:
Si la entrada es texto directo, utilizar el modelo de clasificación de intenciones bancarias basado en texto que ya entrenamos para predecir la intención.


In [ ]:
import re # Import re if not already imported in the notebook
import numpy as np # Import numpy if not already imported

def predict_intention_from_text(text):
    """
    Predicts the banking intention from direct text input.

    Args:
        text (str): The input text from the user.

    Returns:
        str: The predicted banking intention, or an error message.
    """
    # Ensure the necessary models and preprocessors are available
    if 'vectorizer' not in globals() and 'vectorizer' not in locals():
        return "Prediction Error: Text vectorizer model not found. Please ensure it is loaded or trained."
    if 'nn' not in globals() and 'nn' not in locals():
         return "Prediction Error: Text classification model not found. Please ensure it is loaded or trained."
    if 'encoder' not in globals() and 'encoder' not in locals():
        return "Prediction Error: Intention encoder not found. Please ensure it is loaded or trained."
    if 'limpiar_texto' not in globals() and 'limpiar_texto' not in locals():
         return "Prediction Error: Text cleaning function 'limpiar_texto' not found."


    # 1. Clean the input text
    cleaned_text = limpiar_texto(text)

    # 2. Check if the cleaned text is empty
    if not cleaned_text:
        return "No meaningful text found after cleaning."

    # 3. Vectorize the cleaned text
    try:
        X_new = vectorizer.transform([cleaned_text]).toarray()
    except Exception as e:
        return f"Prediction Error during vectorization: {e}"


    # 4. Predict intention
    try:
        predictions = nn.predict(X_new)
    except Exception as e:
        return f"Prediction Error during prediction: {e}"

    # 5. Decode prediction
    try:
        # Ensure the encoder categories are accessible and the prediction index is valid
        if hasattr(encoder, 'categories_') and encoder.categories_ and predictions[0] < len(encoder.categories_[0]):
             predicted_intent = encoder.categories_[0][predictions][0]
        else:
             return f"Prediction Error: Could not decode prediction {predictions[0]}. Invalid index or encoder issue."
    except Exception as e:
        return f"Prediction Error during decoding: {e}"

    # 6. Return the predicted intention
    return predicted_intent

# Example Usage (Optional: for testing the function)
# Assuming vectorizer, nn, encoder, and limpiar_texto are loaded/defined from previous steps
# test_text = "Quiero consultar mi saldo"
# predicted = predict_intention_from_text(test_text)
# print(f"Input Text: '{test_text}' -> Predicted Intent: {predicted}")

# test_text_empty = ""
# predicted_empty = predict_intention_from_text(test_text_empty)
# print(f"Input Text: '{test_text_empty}' -> Predicted Intent: {predicted_empty}")

# test_text_special_chars = "!!! Necesito_información_sobre... un Préstamo?!!!"
# predicted_special = predict_intention_from_text(test_text_special_chars)
# print(f"Input Text: '{test_text_special_chars}' -> Predicted Intent: {predicted_special}")


## Manejar la entrada de imagen

### Subtask:
Implement the logic to handle image input for intention classification. This involves preprocessing the image, performing OCR to extract text, cleaning the extracted text, and then using the appropriate classification model to predict the intention based on the text from the image.


In [ ]:
def predict_intention_from_image(image_path, vectorizer, encoder, neural_network):
    """
    Predicts the banking intention from a document image.

    Args:
        image_path (str): Path to the document image file.
        vectorizer (TfidfVectorizer): Trained TF-IDF vectorizer (fitted on combined OCR data).
        encoder (OneHotEncoder): Trained One-Hot Encoder for intentions (fitted on combined OCR intentions).
        neural_network (NeuralNetwork): Trained neural network model (trained on combined OCR data).

    Returns:
        str: Predicted banking intention.
        str: Extracted and cleaned text from the image.
    """
    # 1. Preprocess the image
    preprocessed_img = preprocess_image_for_ocr(image_path)

    if preprocessed_img is None:
        return "Prediction Error: Could not preprocess image.", ""

    # 2. Extract text with OCR
    try:
        extracted_text = pytesseract.image_to_string(Image.fromarray(preprocessed_img), lang='spa')
    except Exception as e:
        return f"Prediction Error during OCR extraction: {e}", ""


    # 3. Clean the extracted text
    # Use the specific cleaning function for OCR text if different, otherwise use the generic one
    # Assuming limpiar_texto_ocr is available from previous steps
    if 'limpiar_texto_ocr' in globals() or 'limpiar_texto_ocr' in locals():
        cleaned_text = limpiar_texto_ocr(extracted_text)
    elif 'limpiar_texto' in globals() or 'limpiar_texto' in locals():
         cleaned_text = limpiar_texto(extracted_text)
    else:
         return "Prediction Error: Text cleaning function not found.", extracted_text


    if not cleaned_text:
        return "Prediction Error: No meaningful text extracted or cleaned from image.", cleaned_text

    # 4. Vectorize the cleaned text
    try:
        # Use the *fitted* vectorizer to transform the new text
        X_new = vectorizer.transform([cleaned_text]).toarray() # transform expects an iterable
    except Exception as e:
        return f"Prediction Error during vectorization: {e}", cleaned_text

    # 5. Predict intention
    try:
        predictions = neural_network.predict(X_new)
    except Exception as e:
         return f"Prediction Error during neural network prediction: {e}", cleaned_text


    # 6. Decode prediction
    try:
        # Ensure the encoder categories are accessible and the prediction index is valid
        if hasattr(encoder, 'categories_') and encoder.categories_ and predictions[0] < len(encoder.categories_[0]):
             predicted_intent = encoder.categories_[0][predictions][0]
        else:
             return f"Prediction Error: Could not decode prediction {predictions[0]}. Invalid index or encoder issue.", cleaned_text

    except Exception as e:
        return f"Prediction Error during decoding prediction: {e}", cleaned_text

    # 7. Return the predicted intention and the extracted/cleaned text
    return predicted_intent, cleaned_text

print("Defined predict_intention_from_image function.")

In [ ]:
# === Test the predict_intention_from_image function ===
print("Testing the predict_intention_from_image function with simulated images:")

# Get a few simulated document images to test
# Assuming 'documentos_img' directory exists from previous steps
image_files_for_testing = sorted([f for f in os.listdir("documentos_img") if f.endswith(".png")])[:5] # Test with the first 5 images

# Check if necessary models and preprocessors are available (from previous steps)
# We need the models trained on the combined OCR data
required_vars = ['nn_combined_ocr', 'large_ocr_vectorizer', 'combined_ocr_encoder', 'preprocess_image_for_ocr', 'pytesseract', 'Image', 'limpiar_texto_ocr']
all_available = True
for var in required_vars:
    if var not in globals() and var not in locals():
        print(f"Error: Required variable '{var}' not found.")
        all_available = False

if all_available and image_files_for_testing:
    # Use the models trained on the combined OCR data
    current_vectorizer = large_ocr_vectorizer # Assuming large_ocr_vectorizer was updated in Step 7 (Revisited)
    current_encoder = combined_ocr_encoder
    current_nn = nn_combined_ocr


    for image_file in image_files_for_testing:
        image_path = os.path.join("documentos_img", image_file)

        # Call the prediction function
        predicted_intent, extracted_cleaned_text = predict_intention_from_image(image_path, current_vectorizer, current_encoder, current_nn)

        # Display the results
        print(f"\nImage: {image_file}")
        print(f"  Extracted & Cleaned Text: '{extracted_cleaned_text}'")
        print(f"  Predicted Intent: {predicted_intent}")

else:
    print("\nSkipping testing: Required models/preprocessors not available or no test images found.")

## Combinar y decidir la respuesta

### Subtask:
Create a simple logic to determine a basic response based on the predicted banking intention. This logic should map each predicted intention to a predefined response string.


In [ ]:
# Define a mapping from predicted intention to a basic response
intention_responses = {
    "Consultar saldo": "Claro, ¿para qué cuenta deseas consultar el saldo?",
    "Transferir dinero": "Entendido, ¿cuánto y a quién deseas transferir dinero?",
    "Pagar tarjeta de crédito": "Puedo ayudarte con el pago de tu tarjeta de crédito. ¿Cuál deseas pagar y cuánto?",
    "Abrir cuenta": "Para abrir una nueva cuenta, necesito algunos datos. ¿Qué tipo de cuenta te interesa?",
    "Cerrar cuenta": "Comprendo que deseas cerrar tu cuenta. ¿Podrías indicar el motivo?",
    "Solicitar préstamo": "Puedo darte información sobre nuestros préstamos. ¿Qué tipo de préstamo te interesa (personal, hipotecario, etc.)?",
    "Bloquear tarjeta": "Si tu tarjeta ha sido robada o perdida, puedo ayudarte a bloquearla inmediatamente. ¿Confirmas el bloqueo?",
    "Actualizar datos personales": "Para actualizar tus datos personales, te guiaré a través del proceso de verificación. ¿Estás listo?",
    "Consultar movimientos": "Claro, ¿de qué cuenta y período deseas consultar los movimientos?",
    "Reportar transacción sospechosa": "Lamento escuchar eso. Por favor, describe la transacción sospechosa.",
    "Cheque": "Detecté un cheque. ¿Necesitas procesarlo o alguna información sobre él?",
    "Estado de cuenta": "Detecté un estado de cuenta. ¿Necesitas consultarlo o realizar alguna acción basada en él?",
    "Recibo de depósito": "Detecté un recibo de depósito. ¿Quieres confirmar la transacción o necesitas más detalles?",
    "Comprobante de transferencia": "Detecté un comprobante de transferencia. ¿Necesitas verificarla o tienes alguna pregunta?",
    # Add a default response for any unhandled intentions
    "Unknown Intent": "No estoy seguro de cómo ayudarte con eso. ¿Podrías reformular tu solicitud?",
    "No meaningful text found after cleaning": "No pude extraer texto claro de la entrada. ¿Puedes intentarlo de nuevo con un texto más claro?",
    "Prediction Error: Could not preprocess image.": "Hubo un error al procesar la imagen. Por favor, asegúrate de que el archivo de imagen sea válido.",
    "Prediction Error during OCR extraction:": "Hubo un error al extraer texto de la imagen. El documento podría no ser legible.",
    "Prediction Error: No meaningful text extracted or cleaned from image.": "No se pudo extraer texto significativo de la imagen. Por favor, intenta con un documento diferente.",
    "Prediction Error during vectorization:": "Ocurrió un error al procesar el texto para la predicción.",
    "Prediction Error during neural network prediction:": "Ocurrió un error al realizar la predicción con el modelo.",
    "Prediction Error during decoding prediction:": "Ocurrió un error al interpretar la predicción.",
    "Prediction Error: Could not decode prediction": "Ocurrió un error al decodificar la predicción.",
    "Prediction Error: Text vectorizer model not found.": "Error: El modelo de vectorización de texto no está disponible.",
    "Prediction Error: Text classification model not found.": "Error: El modelo de clasificación de texto no está disponible.",
    "Prediction Error: Intention encoder not found.": "Error: El codificador de intenciones no está disponible.",
    "Prediction Error: Text cleaning function 'limpiar_texto' not found.": "Error: La función de limpieza de texto no está disponible."


}

print("Defined intention_responses mapping.")

## Implementar una función de entrada unificada

### Subtask:
Create a main function that can accept input (either text or image path) and a parameter indicating the input type ('text' or 'image'), and then route the processing to the appropriate function (`predict_intention_from_text` or `predict_intention_from_image_combined`) based on the input type. This function should return the predicted intention and the corresponding basic response.


In [ ]:
def process_input(input_data, input_type):
    """
    Processes input (text or image) and predicts the banking intention, returning a basic response.

    Args:
        input_data (str): The input data, either direct text or the path to an image file.
        input_type (str): The type of input, either 'text' or 'image'.

    Returns:
        tuple: A tuple containing the predicted intention (str) and the corresponding response (str).
               Returns error messages for invalid input types or processing failures.
    """
    predicted_intention = "Unknown Intent" # Default intention

    if input_type == 'text':
        # Use the text prediction function
        predicted_intention = predict_intention_from_text(input_data)
        # If predict_intention_from_text returns an error string, use it as the intention
        if predicted_intention.startswith("Prediction Error") or predicted_intention.startswith("No meaningful text found"):
            response = intention_responses.get(predicted_intention, intention_responses["Unknown Intent"])
            return predicted_intention, response
        extracted_text = input_data # For text input, extracted text is the input itself (or cleaned)

    elif input_type == 'image':
        # Ensure necessary models for image processing are available
        if ('large_ocr_vectorizer' not in locals() and 'large_ocr_vectorizer' not in globals()) and \
           ('combined_ocr_vectorizer' not in locals() and 'combined_ocr_vectorizer' not in globals()):
            return "Prediction Error: OCR vectorizer model not found.", intention_responses.get("Prediction Error: OCR vectorizer model not found.", intention_responses["Unknown Intent"])
        if 'combined_ocr_encoder' not in locals() and 'combined_ocr_encoder' not in globals():
            return "Prediction Error: OCR encoder not found.", intention_responses.get("Prediction Error: OCR encoder not found.", intention_responses["Unknown Intent"])
        if 'nn_combined_ocr' not in locals() and 'nn_combined_ocr' not in globals():
            return "Prediction Error: Combined NN model not found.", intention_responses.get("Prediction Error: Combined NN model not found.", intention_responses["Unknown Intent"])
        if 'predict_intention_from_image' not in locals() and 'predict_intention_from_image' not in globals():
             return "Prediction Error: Image prediction function not found.", intention_responses["Unknown Intent"]


        # Determine the correct vectorizer variable to use
        current_ocr_vectorizer = large_ocr_vectorizer if 'large_ocr_vectorizer' in locals() or 'large_ocr_vectorizer' in globals() else combined_ocr_vectorizer


        # Use the image prediction function
        predicted_intention, extracted_text = predict_intention_from_image(input_data, current_ocr_vectorizer, combined_ocr_encoder, nn_combined_ocr)

        # If predict_intention_from_image returns an error string as the intention, use it
        if predicted_intention.startswith("Prediction Error"):
            response = intention_responses.get(predicted_intention, intention_responses["Unknown Intent"])
            return predicted_intention, response

    else:
        return "Invalid input type specified.", intention_responses["Unknown Intent"]

    # Look up the basic response based on the predicted intention
    # Use the get() method with a default value to handle unseen intentions gracefully
    response = intention_responses.get(predicted_intention, intention_responses["Unknown Intent"])

    return predicted_intention, response

print("Defined process_input function.")

# === Optional: Test the process_input function ===
print("\nTesting the end-to-end process_input pipeline:")

# Test with text input
test_text_input = "Quiero saber mi saldo"
predicted_intent_text, response_text = process_input(test_text_input, 'text')
print(f"Input Type: Text, Input: '{test_text_input}'")
print(f"  Predicted Intent: {predicted_intent_text}")
print(f"  Response: {response_text}")


# Test with image input (use a sample image from the generated ones)
# Assuming 'documentos_img' directory and image files exist from previous steps
image_files_for_testing_process = sorted([f for f in os.listdir("documentos_img") if f.endswith(".png")])
if image_files_for_testing_process:
    test_image_path = os.path.join("documentos_img", image_files_for_testing_process[0])
    predicted_intent_image, response_image = process_input(test_image_path, 'image')
    print(f"\nInput Type: Image, Input: '{test_image_path}'")
    print(f"  Predicted Intent: {predicted_intent_image}")
    print(f"  Response: {response_image}")
else:
    print("\nNo image files found in 'documentos_img' for testing image input.")


# Test with invalid input type
predicted_intent_invalid, response_invalid = process_input("some data", 'video')
print(f"\nInput Type: Invalid (video), Input: 'some data'")
print(f"  Predicted Intent: {predicted_intent_invalid}")
print(f"  Response: {response_invalid}")

## Probar la integración multimodal

### Subtask:
Test the main `process_input` function with examples of both text and image input to verify that both paths of the multimodal pipeline are working correctly and producing predicted intentions and responses.


In [ ]:
# === Test the main process_input function with examples ===
print("--- Testing the End-to-End Multimodal Pipeline (process_input) ---")

# Test Case 1: Text Input
test_text_input = "Quiero consultar mi saldo"
print(f"\nTesting with Text Input: '{test_text_input}'")
predicted_intent_text, response_text = process_input(test_text_input, 'text')
print(f"  Predicted Intent: {predicted_intent_text}")
print(f"  Response: {response_text}")


# Test Case 2: Image Input
# Assuming 'documentos_img' directory and image files exist from previous steps
image_files_for_testing_process = sorted([f for f in os.listdir("documentos_img") if f.endswith(".png")])
if image_files_for_testing_process:
    test_image_path = os.path.join("documentos_img", image_files_for_testing_process[0])
    print(f"\nTesting with Image Input: '{test_image_path}'")
    predicted_intent_image, response_image = process_input(test_image_path, 'image')
    print(f"  Predicted Intent: {predicted_intent_image}")
    print(f"  Response: {response_image}")
else:
    print("\nSkipping Image Input Test: No image files found in 'documentos_img'.")


# Test Case 3: Invalid Input Type
test_invalid_input = "some random data"
test_invalid_type = "audio"
print(f"\nTesting with Invalid Input Type: '{test_invalid_type}'")
predicted_intent_invalid, response_invalid = process_input(test_invalid_input, test_invalid_type)
print(f"  Predicted Intent: {predicted_intent_invalid}")
print(f"  Response: {response_invalid}")

print("\n--- End of Multimodal Pipeline Tests ---")

## Resumen de la Integración Multimodal

### Hallazgos Clave del Análisis de Datos (y Pipeline Implementada)

*   La pipeline diseñada puede aceptar dos tipos principales de entrada: **texto directo** e **imágenes de documentos**.
*   Se implementó una función (`predict_intention_from_text`) para manejar la entrada de texto, utilizando el modelo de clasificación de texto previamente entrenado para predecir la intención bancaria.
*   Se implementó una función (`predict_intention_from_image`) para manejar la entrada de imagen, que incorpora el preprocesamiento de la imagen, la realización de OCR para extraer texto, la limpieza del texto extraído y luego el uso del modelo de clasificación entrenado con datos de OCR para predecir la intención basándose en el texto de la imagen.
*   Se creó un diccionario (`intention_responses`) para mapear las intenciones predichas (incluyendo tipos de documento como 'Cheque' y 'Estado de cuenta' que se derivan del OCR) y posibles mensajes de error a respuestas básicas predefinidas.
*   Se creó una función unificada (`process_input`) como punto de entrada principal, que dirige la entrada a la función de predicción adecuada según el tipo de entrada especificado ('text' o 'image') y recupera la respuesta básica correspondiente.
*   Las pruebas confirmaron que la función `process_input` maneja correctamente tanto las entradas de texto como las de imagen, demostrando ambos caminos de la pipeline multimodal. (Aunque se encontraron y resolvieron errores durante la implementación).
*   El camino de entrada de imagen logró extraer texto, predecir la intención (por ejemplo, "Recibo de depósito" para una imagen de cheque de ejemplo) y proporcionar una respuesta.
*   Se validó el manejo de errores para tipos de entrada no válidos en `process_input`.

### Ideas o Próximos Pasos

*   Mejorar la lógica de generación de respuestas para que sean más dinámicas y conscientes del contexto, basándose en la intención predicha y, potencialmente, en entidades extraídas del texto o la imagen de entrada.
*   Expandir el módulo de visión para incluir la clasificación del *tipo* de documento (Cheque, Estado de Cuenta, etc.) utilizando técnicas de visión por computadora y entrenar un modelo específico para esta tarea.
*   Considerar la extracción de *entidades* (como montos, fechas, nombres) del texto extraído por OCR para enriquecer la información disponible para la gestión del diálogo o la generación de respuestas.
*   Desarrollar una gestión de diálogo más sofisticada que mantenga un estado y memoria conversacional para manejar interacciones de varios turnos.

In [ ]:
# Clear Hugging Face cache directories
import os
import shutil

# Get the default cache directory
# This might vary slightly based on environment, but this is common
hf_cache_home = os.path.join(os.path.expanduser("~"), ".cache", "huggingface")

if os.path.exists(hf_cache_home):
    print(f"Clearing Hugging Face cache directory: {hf_cache_home}")
    try:
        shutil.rmtree(hf_cache_home)
        print("Cache cleared successfully.")
    except Exception as e:
        print(f"Error clearing cache: {e}")
else:
    print(f"Hugging Face cache directory not found at {hf_cache_home}. No cache to clear.")

# Note: You might need to restart the runtime again after clearing the cache,
# although clearing it programmatically often makes it effective immediately for new downloads.
# It's safer to restart and then re-run the setup and model loading.

In [ ]:
!pip install tf-keras

In [ ]:
!pip install tensorflow

Módulo NLU con BERT

In [ ]:
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
import tensorflow as tf # Import TensorFlow

modelo_name = "bert-base-multilingual-cased"
num_clases = 3  # por ejemplo, número de etiquetas

tokenizer = AutoTokenizer.from_pretrained(modelo_name)
# from_pt=True indica que cargue los pesos desde un checkpoint de PyTorch si está disponible
model = TFAutoModelForSequenceClassification.from_pretrained(modelo_name, num_labels=num_clases, from_pt=True)

In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import tensorflow as tf

modelo_name = "bert-base-multilingual-cased"  # Soporta español y otros idiomas

# Define the number of classes based on your banking intentions
intentos_o_clases = 10 # Assuming 10 classes based on the original dataset


tokenizer = AutoTokenizer.from_pretrained(modelo_name)
# Try to explicitly tell the library not to load from PyTorch checkpoint
model = TFAutoModelForSequenceClassification.from_pretrained(modelo_name, num_labels=intentos_o_clases, from_pt=True)

def predecir_intencion(texto):
    inputs = tokenizer(texto, return_tensors="tf", truncation=True, padding=True)
    outputs = model(inputs)
    logits = outputs.logits
    pred = tf.argmax(logits, axis=1).numpy()[0]
    return pred

Módulo Visión
OCR con pytesseract

In [ ]:
import pytesseract
import cv2

def extraer_texto(imagen_path):
    img = cv2.imread(imagen_path)
    gris = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, umbral = cv2.threshold(gris, 150, 255, cv2.THRESH_BINARY)
    texto = pytesseract.image_to_string(umbral, lang='spa')
    return texto


Clasificación de documentos

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224,224,3))
x = GlobalAveragePooling2D()(base_model.output)
output = Dense(num_clases, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=output)

# Congelar base_model si quieres solo entrenar la cabeza
base_model.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


Gestión de diálogo

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

chat_history_ids = None

def responder_usuario(texto_usuario):
    global chat_history_ids
    new_input_ids = tokenizer.encode(texto_usuario + tokenizer.eos_token, return_tensors='pt')
    bot_input_ids = new_input_ids if chat_history_ids is None else torch.cat([chat_history_ids, new_input_ids], dim=-1)
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    respuesta = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    return respuesta


Interfaz Web y API
Con FastAPI (recomendado para API y web rápido):

In [ ]:
!pip install python-multipart

In [ ]:
!pip install fastapi

In [ ]:
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import JSONResponse

app = FastAPI()

@app.post("/procesar_documento/")
async def procesar_documento(file: UploadFile = File(...)):
    # Guardar archivo temporalmente
    contenido = await file.read()
    with open("temp.png", "wb") as f:
        f.write(contenido)
    texto = extraer_texto("temp.png")  # usa función OCR antes definida
    # Aquí agregas llamada a clasificador, NLU, diálogo, etc.
    return JSONResponse({"texto_extraido": texto})

# Ejecutar con: uvicorn nombre_script:app --reload


## Instalar fastapi y uvicorn

In [ ]:
# Install FastAPI and Uvicorn
!pip install fastapi uvicorn

## Instalar y configurar ngrok

In [ ]:
import time
import os
from pyngrok import ngrok

# 1. Configure your ngrok authentication token
# *** IMPORTANT ***: Replace "YOUR_NGROK_AUTH_TOKEN" with your actual ngrok auth token.
# You can get your token from your ngrok dashboard after signing up at ngrok.com
NGROK_AUTH_TOKEN = "30vg34wt2hnWqIxx7EyI4MxLfEk_tLv3MfwiJ4xwh9JqY1RS" # <-- Replace this!

if NGROK_AUTH_TOKEN == "YOUR_NGROK_AUTH_TOKEN":
    print("Please replace 'YOUR_NGROK_AUTH_TOKEN' with your actual ngrok auth token.")

else:
    try:
        # Set the auth token
        ngrok.set_auth_token(NGROK_AUTH_TOKEN)
        print("ngrok authentication token set.")

        # 2. Define the port Uvicorn is running on (from previous step)
        port = 8000 # Ensure this matches the port used by Uvicorn

        # 3. Create an ngrok tunnel to the specified port
        # Disconnect any existing tunnels first to avoid conflicts
        ngrok.kill()
        # Create a new http tunnel
        # Use bind_tls=True for https tunnel as required by some browsers/APIs
        public_url = ngrok.connect(port, bind_tls=True).public_url

        # 4. Print the public URL provided by ngrok
        if public_url:
            print(f"\nFastAPI application is accessible at: {public_url}")

            # 5. Test the public URL using curl (GET request to root endpoint)
            print("\nTesting the public URL with curl...")
            # Use !curl to run shell command in Colab
            # Note: For POST requests to /process_input/, you would need a more complex curl command or use requests library
            !curl -X GET $public_url

        else:
            print("\nFailed to create ngrok tunnel. The FastAPI application might not be publicly accessible.")

    except Exception as e:
        print(f"An error occurred during ngrok setup or testing: {e}")

    # 6. Keep the cell running to keep the server and tunnel active
    # The Uvicorn server is assumed to be running in a separate thread from the previous cell.
    # This cell needs to stay alive for the ngrok tunnel to persist.
    print("\nKeep this cell running to keep the ngrok tunnel active.")
    # Add a long sleep or similar mechanism if needed to prevent the cell from finishing prematurely
    # time.sleep(3600) # Keep alive for 1 hour (Optional, depending on Colab's behavior)

In [ ]:
from fastapi import FastAPI

# Instantiate a FastAPI application
app = FastAPI()

# Define a root endpoint that responds to GET requests
@app.get("/")
def read_root():
    return {"message": "Hello from FastAPI in Colab!"}

print("FastAPI app instance created with a root endpoint.")

In [ ]:
from fastapi import FastAPI, UploadFile, File, Form
from fastapi.responses import JSONResponse
import os

# Assuming the following functions/variables from previous steps are available:
# - process_input(input_data, input_type)
# - predict_intention_from_text(text)
# - predict_intention_from_image(image_path, vectorizer, encoder, neural_network)
# - intention_responses (dictionary)
# - vectorizer (for text)
# - nn (for text)
# - encoder (for text)
# - large_ocr_vectorizer or combined_ocr_vectorizer (for image)
# - nn_combined_ocr (for image)
# - combined_ocr_encoder (for image)
# - limpiar_texto (for text cleaning)
# - preprocess_image_for_ocr (for image preprocessing)
# - pytesseract (for OCR)
# - Image (from PIL)
# - limpiar_texto_ocr (for OCR text cleaning)


app = FastAPI()

# Define the root endpoint (optional, keep for basic check)
@app.get("/")
def read_root():
    return {"message": "API is running. Use the /process_input/ endpoint for predictions."}

# Define a new endpoint to handle multimodal input
@app.post("/process_input/")
async def process_multimodal_input(
    input_type: str = Form(...), # 'text' or 'image'
    text_input: str = Form(None), # Optional text input
    image_file: UploadFile = File(None) # Optional image file input
):
    """
    Processes either text or image input and predicts the banking intention.

    Args:
        input_type (str): Specifies the type of input ('text' or 'image').
        text_input (str, optional): The direct text input if input_type is 'text'.
        image_file (UploadFile, optional): The uploaded image file if input_type is 'image'.

    Returns:
        JSONResponse: A JSON object containing the predicted intention and response.
    """
    predicted_intent = "Unknown Intent"
    response = intention_responses.get("Unknown Intent", "No estoy seguro de cómo ayudarte con eso.")
    extracted_text = "" # To store extracted text from image if applicable

    try:
        if input_type == 'text':
            if text_input is None:
                return JSONResponse(status_code=400, content={"error": "Text input is required for input_type 'text'."})

            # Use the text prediction function
            predicted_intent, response = process_input(text_input, 'text')
            extracted_text = text_input # For text input, the "extracted" text is the input

        elif input_type == 'image':
            if image_file is None:
                 return JSONResponse(status_code=400, content={"error": "Image file is required for input_type 'image'."})

            # Save the uploaded image temporarily
            temp_image_path = "temp_uploaded_image.png"
            with open(temp_image_path, "wb") as f:
                f.write(await image_file.read())

            # Use the image prediction function
            # Ensure the correct vectorizer is used (large_ocr_vectorizer or combined_ocr_vectorizer)
            current_ocr_vectorizer = large_ocr_vectorizer if 'large_ocr_vectorizer' in locals() or 'large_ocr_vectorizer' in globals() else (combined_ocr_vectorizer if 'combined_ocr_vectorizer' in locals() or 'combined_ocr_vectorizer' in globals() else None)

            if current_ocr_vectorizer and 'combined_ocr_encoder' in globals() or 'combined_ocr_encoder' in locals() and 'nn_combined_ocr' in globals() or 'nn_combined_ocr' in locals():
                 predicted_intent, extracted_text = predict_intention_from_image(temp_image_path, current_ocr_vectorizer, combined_ocr_encoder, nn_combined_ocr)
            else:
                 predicted_intent = "Prediction Error: Required models for image processing not found."
                 extracted_text = ""


            # Clean up the temporary file
            if os.path.exists(temp_image_path):
                os.remove(temp_image_path)

            # If the prediction function returned an error message as intent, use it for response
            if predicted_intent.startswith("Prediction Error"):
                 response = intention_responses.get(predicted_intent, intention_responses["Unknown Intent"])
            else:
                # Look up the response based on the predicted intention from image
                response = intention_responses.get(predicted_intent, intention_responses["Unknown Intent"])


        else:
            return JSONResponse(status_code=400, content={"error": "Invalid input_type. Use 'text' or 'image'."})

        # Return the predicted intention and response
        return JSONResponse(content={
            "input_type": input_type,
            "extracted_text": extracted_text, # Include extracted text for context
            "predicted_intent": predicted_intent,
            "response": response
        })

    except Exception as e:
        # Catch any unexpected errors during processing
        return JSONResponse(status_code=500, content={"error": f"An internal error occurred: {e}"})

print("FastAPI endpoint /process_input/ defined and integrated with multimodal pipeline functions.")

In [ ]:
import asyncio
from uvicorn import Config, Server
from pyngrok import ngrok
import threading
import time

# 4. Define the host and port for the Uvicorn server
host = "0.0.0.0"
port = 8000

# 5. Create an asyncio event loop
loop = asyncio.get_event_loop()

# 6. Create a uvicorn.Config object for the FastAPI app instance
# Assuming the 'app' FastAPI instance is available from a previous cell
config = Config(app=app, host=host, port=port, loop=loop)

# 7. Create a uvicorn.Server instance with the created config
server = Server(config=config)

# Function to run the server in a separate thread
def run_server():
    loop.run_until_complete(server.serve())

# 8. Start the Uvicorn server in a separate thread
# Check if the loop is already running, if so, run the server directly
# Otherwise, use a separate thread
if loop.is_running():
    print("Running server in the current loop.")
    # Use run_coroutine_threadsafe to submit the coroutine to the running loop
    asyncio.run_coroutine_threadsafe(server.serve(), loop)
else:
    print("Running server in a new thread.")
    # Use a separate thread to run the loop and the server
    server_thread = threading.Thread(target=run_server)
    server_thread.start()

# 9. Create an ngrok tunnel to the specified port
try:
    # Disconnect any existing tunnels first
    ngrok.kill()
    # Create a new http tunnel
    public_url = ngrok.connect(port, bind_tls=True).public_url
    # ngrok.connect returns a NgrokTunnel object, access the public_url attribute
except Exception as e:
    print(f"Error creating ngrok tunnel: {e}")
    public_url = None

# 10. Print the public URL provided by ngrok
if public_url:
    print(f"\nFastAPI application is accessible at: {public_url}")
else:
    print("\nFailed to create ngrok tunnel. The FastAPI application might not be publicly accessible.")


# 11. Keep the cell running to keep the server and tunnel active
# You might need to add a placeholder like `time.sleep(3600)` in a real scenario
# in interactive Colab to keep the cell running indefinitely, or rely on the
# server thread keeping the process alive.
# For demonstration, we'll print a message and the cell will remain active
# until manually stopped or the Colab session ends.

print("\nKeep this cell running to keep the FastAPI server and ngrok tunnel active.")

# Optional: Add a long sleep if needed to prevent the cell from finishing prematurely
# time.sleep(3600) # Keep alive for 1 hour

In [ ]:
from pyngrok import ngrok
print("Intentando detener procesos ngrok existentes inmediatamente antes de iniciar el túnel...")
try:
    ngrok.kill()
    print("Procesos ngrok detenidos.")
except Exception as e:
    print(f"Error al intentar detener procesos ngrok: {e}")